In [ ]:
import os, sys, json, time, cv2, skimage
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

# 수정 필요 : "./dpc/" 에 대한 direct path  ,  만약 console이 dpc 폴더에 위치한다면 ROOT_DIR = os.getcwd()
ROOT_DIR = '/content/gdrive/MyDrive/new_mrcnn/Mask_RCNN/'
sys.path.append(ROOT_DIR)

# 수정 필요 dpc 폴더 하위에 log 폴더 존재시 아래와 같이 가능
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# 수정 필요 COCO_MODEL_PATH = 다운로드받은 pretrained weight의 path
COCO_MODEL_PATH = os.path.join('/content/gdrive/MyDrive/new_mrcnn/Mask_RCNN/logs/ai_dpc20230309T0127/', "mask_rcnn_ai_dpc_0010.h5")

class InferenceConfig(Config):
    NAME = "AI_DPC"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    LEARNING_RATE = 0.001
    NUM_CLASSES = 1+1
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    BACKBONE = 'resnet101'
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000
    DETECTION_MIN_CONFIDENCE = 0.8

inference_config = InferenceConfig()

test_model = modellib.MaskRCNN(
    mode = 'inference',
    config = inference_config,
    # 수정 필요 COCO_MODEL_PATH의 상위 디렉토리
    model_dir = '/content/gdrive/MyDrive/new_mrcnn/Mask_RCNN/logs/ai_dpc20230309T0127/'
)
# 수정 필요 COCO_MODEL_PATH = 다운로드받은 pretrained weight의 path
# trained_weight_path = COCO_MODEL_PATH
trained_weight_path = os.path.join('/content/gdrive/MyDrive/new_mrcnn/Mask_RCNN/logs/ai_dpc20230309T0127/', "mask_rcnn_ai_dpc_0010.h5")

test_model.load_weights(trained_weight_path, by_name = True)

mask_colors = [
    (0., 0., 0.),
    (0., 1., 0.)
]

# 수정 필요 Inference할 이미지들이 존재하는 directory path
real_test_dir = '/content/gdrive/MyDrive/Mask_RCNN/test_images/'
image_paths = []
class_names = ['BG', 'problem']

# 수정 필요 Inference 결과 저장할 directory path
save_path = "/content/gdrive/MyDrive/new_mrcnn/test6/"

for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg']:
        image_paths.append(os.path.join(real_test_dir, filename))

for image_path in image_paths:
    save_name = image_path.split('/')[-1]
    img = skimage.io.imread(image_path)
    img_arr = np.array(img)
    
    results = test_model.detect([img_arr], verbose = 1)
    r = results[0]
    colors = tuple(np.take(mask_colors, r['class_ids'], axis = 0))
    save_json = {
        "file_name" : save_name,
        "bboxes" : r['rois'].tolist(),
    }
    # 수정필요 encoding = 'utf-8' or 'cp949' 등 , bounding box가 도식화된 이미지를 저장하고 싶지 않다면 pass
    with open(save_path+"/{}.json".format(save_name[:-4]), "w") as json_file:
        json.dump(save_json, json_file)
    
    img_copy = img.copy()
    for bbox in r['rois']:
        y1, x1, y2, x2 = bbox
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 3)
    visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'],
                                class_names, r['scores'], figsize = (16, 8),
                                show_bbox = True, colors = colors)
    
    # 수정필요 bounding box가 도식화된 이미지를 저장하고 싶지 않다면 pass
    plt.imsave(save_path+'/{}'.format(save_name), img_copy, dpi = 300)